In [77]:
import csv
import random
import pickle
import numpy as np
import itertools
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt

import pandas as pd

In [14]:
# Load and clean data
useract = pd.read_csv("user_activities.csv")
useract = useract.drop(columns=['id'])
useract = useract.dropna(how='all')

useract

,interests,graduated_courses,on_progress_courses,events_attended,challenge_joined,winning_apps
0,NaN,NaN,"Bangkit Android Assessment,Bangkit General Ass...",NaN,NaN,NaN
1,NaN,NaN,Belajar Membuat Aplikasi Android untuk Pemula,NaN,NaN,NaN
2,NaN,"Menjadi Game Developer Expert,Belajar Membuat ...",NaN,"BEKRAF Developer Day 2019 - Malang,Baparekraf ...",#MakanTanpaSisa Instagram AR Filter Competitio...,"Quiz Ramadhan,Ngabuburit BS (Benar Salah),Tari..."
3,NaN,NaN,Memulai Pemrograman Dengan Java,NaN,NaN,NaN
4,NaN,NaN,Belajar Membuat Aplikasi Android untuk Pemula,NaN,NaN,NaN
...,...,...,...,...,...,...
994,NaN,NaN,"Bangkit Application Screening,Belajar Fundamen...",NaN,NaN,NaN
996,NaN,NaN,"Belajar Membuat Aplikasi Android untuk Pemula,...",NaN,NaN,NaN
997,NaN,NaN,"Belajar Dasar Pemrograman Web,Belajar Membuat ...",BEKRAF Developer Day 2019 - Malang,NaN,NaN
998,NaN,NaN,"Belajar Dasar Pemrograman JavaScript,Belajar M...",NaN,NaN,NaN


In [113]:
# TODO: Coursesnya ga lengkap anjeng

courses = pd.read_csv("courses.csv")
coursesname = courses["name"].tolist()

In [107]:
# Aggregate data, merge all valid entries into a list

def mergeall(rowdata):
    aggregated_datas = ",".join([str(s) for s in list(rowdata.values) if not pd.isna(s)])
    return aggregated_datas.split(",")

def mergecourse(rowdata):
    aggregated_datas = ",".join([str(s) for s in [rowdata.graduated_courses, rowdata.on_progress_courses] if not pd.isna(s)])
    return aggregated_datas.split(",")

merged_datas = list(useract.apply(mergeall, axis = 1))
merged_courses = list(useract.apply(mergecourse, axis = 1))

In [108]:
# From entries, one course is tagged as the "output" while the rest is used as input.
# So if one profile has several associated courses, it can be augmented into several training entries.

agumented_datas = {
    'x': [],
    'y': []
}

for i in range(len(merged_datas)):
    merged_data = merged_datas[i]
    merged_course = merged_courses[i]
    
    for course in merged_course:
        #Simpen dulu buat nanti kalo mau ditambahin ingfo non-courses
        #agumented_datas['x'].append([data for data in merged_data if data != course])
        agumented_datas['x'].append([data for data in merged_course if data != course])
        agumented_datas['y'].append(course)

#No need to dataframe-ize it for now       
#augmented_df = pd.DataFrame(agumented_datas)
#augmented_df
agumented_datas['x'][6]

['Menjadi Game Developer Expert',
 'Belajar Membuat Game untuk Pemula',
 'Membangun Progressive Web Apps',
 'Belajar Dasar Pemrograman Web',
 'Belajar Fundamental Front-End Web Development',
 'Menjadi Front-End Web Developer Expert',
 'Cloud Practitioner Essentials (Belajar Dasar AWS Cloud)',
 'Belajar Dasar Pemrograman JavaScript',
 'Belajar Membuat Aplikasi Back-End untuk Pemula',
 'Belajar Membuat Augmented Reality dengan Lens Studio']

In [115]:
# GRADED FUNCTION: fit_tokenizer
def get_tokenizer_dict(train_list):
    return dict(zip(train_list, [i+1 for i in range(len(train_list))]))

def tokenize(tokenizer_dict, corpus):
    output = []
    for line in corpus:
        tokenizedline = [tokenizer_dict.get(entry, 0) for entry in line]
        output.append(tokenizedline)
    
    return output

def pad_tokens(input_sequences, maxlen):
    return np.array(pad_sequences(input_sequences, maxlen=maxlen, padding='pre'))

tokenizer_dict = get_tokenizer_dict(coursesname)
agumented_datas['x_tokens'] = pad_tokens(tokenize(tokenizer_dict, agumented_datas["x"]), 5)
agumented_datas['x_tokens']


array([[ 0,  0,  0,  9,  1],
       [ 0,  0,  0,  9,  1],
       [ 0,  0,  0,  0,  1],
       ...,
       [ 0, 30, 31,  0,  7],
       [ 0, 30, 31,  0, 29],
       [ 0,  0,  0,  0,  0]], dtype=int32)

In [66]:
print(augmented_df.iloc[1]["x"])
seq = tokenizer.texts_to_sequences(augmented_df.iloc[1]["x"])
padseq = np.array(pad_sequences(seq, maxlen=10))
padseq

['Bangkit Android Assessment', 'Belajar Dasar Pemrograman Web', 'Belajar Fundamental Aplikasi Android']


array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 11],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  3]], dtype=int32)

In [53]:
# Masi Ngasal
def create_model(vocab_size, embedding_dim, maxlen, embeddings_matrix):
    
    ### START CODE HERE
    
    model_userprefs = tf.keras.Sequential([ 
        tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=maxlen, weights=[embeddings_matrix], trainable=False),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    model_courseord = tf.keras.Sequential([ 
        tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=maxlen, weights=[embeddings_matrix], trainable=False),
        tf.keras.layers.Bidirectional(tf.keras.layers.GRU(16)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])
    
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy']) 

    ### END CODE HERE

    return model